In [ ]:
!pip install llama-index llama-index-readers-web duckduckgo_search

# Identify specific industry(s) that the user input belongs to (used to match industry to corresponding data regulation policies)

### Define locations and link to regulations

In [3]:
import json
import pandas as pd

In [4]:
data = pd.read_csv('/content/Industry Data Regulation.csv')

In [5]:
data

,regulations,industry,regions,links
0,COPPA,Children,USA,https://www.ftc.gov/legal-library/browse/rules...
1,HIPAA,Medical,USA,https://www.cdc.gov/phlp/php/resources/health-...
2,GLBA,Finance,USA,https://www.ftc.gov/business-guidance/privacy-...
3,FCRA,Consumption,USA,https://www.ftc.gov/legal-library/browse/statu...
4,FERPA,Education,USA,https://www.disabilityrightssc.org/wp-content/...
5,RFPA,Finance,USA,https://www.fdic.gov/resources/supervision-and...
6,VPPA,Digital Media,USA,https://www.congress.gov/bill/100th-congress/s...
7,ECPA,Communication,USA,https://bja.ojp.gov/program/it/privacy-civil-l...


### LLM (identify industry)

In [8]:
import os

import regex as re
import pandas as pd
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.core import SummaryIndex, VectorStoreIndex
from llama_index.readers.web import SimpleWebPageReader
from llama_index.core.tools import BaseTool, FunctionTool
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.agent import ReActAgent
from IPython.display import Markdown, display
from duckduckgo_search import DDGS
os.environ["OPENAI_API_KEY"] = "sk-proj-e20Q6bKuRvBVQBObIFhvT3BlbkFJ3HdudAgJER9AnPxwsgFp"

In [9]:
token = 'sk-proj-e20Q6bKuRvBVQBObIFhvT3BlbkFJ3HdudAgJER9AnPxwsgFp'
llm = OpenAI(model="gpt-4o", openai_api_key=token)
Settings.llm = llm

In [20]:
industry_names = ', '.join(list(data['industry']))
INDUSTRY_TEMPLATE = """
    Assistant: I can help determine which industry a user input belongs to. The options are: {industries}, Other. Make sure to include all options.
    User: {user_input}
    Assistant: The industry that the user input belongs to is (are):
    Example:
    User: children
    Assistant: Children
    User: consume
    Assistant: Consumption
    User: school
    Assistant: Education
    User: media
    Assistant: Digital Media
    Now, let's try with your input.
    User: {user_input}
    Assistant: The industry that the user input belongs to is (are):
    """

In [22]:
def determine_industry(user_input, data = data, api_key = "sk-proj-e20Q6bKuRvBVQBObIFhvT3BlbkFJ3HdudAgJER9AnPxwsgFp", temperature=0.3):
    # Initialize the OpenAI model with llama-index
    llm = OpenAI(model="gpt-4o", openai_api_key=api_key)

    industry_names = ', '.join(list(data['industry']))

    INDUSTRY_TEMPLATE = """
    Assistant: I can help determine which industry a user input belongs to. The options are: {industries}, Other. Make sure to include all options.
    User: {user_input}
    Assistant: The industry that the user input belongs to is (are):
    Example:
    User: children
    Assistant: Children
    User: consume
    Assistant: Consumption
    User: school
    Assistant: Education
    User: media
    Assistant: Digital Media
    Now, let's try with your input.
    User: {user_input}
    Assistant: The industry that the user input belongs to is (are):
    """

    prompt = INDUSTRY_TEMPLATE.format(industries=industry_names, user_input=user_input)

    response = llm.complete(prompt=prompt, max_tokens=50, temperature=temperature)

    answer = response.text.strip().replace("The industry that the user input belongs to is (are):", "").strip()

    return answer

In [43]:
def clean_output(text):
  return text.split('\n')[0]

In [29]:
def get_data_regulation_url(data, industry_name):
  return data[data['industry'] == industry_name]['links'].tolist()

In [44]:
def get_industry_regulation_links(user_input, data = data, api_key = "sk-proj-e20Q6bKuRvBVQBObIFhvT3BlbkFJ3HdudAgJER9AnPxwsgFp"):
    industry = determine_industry(user_input, data, api_key)
    industry = clean_output(industry)
    industry_list = industry.split(', ') # Assuming regions are comma-separated
    links = []
    for industry in industry_list:
        link = get_data_regulation_url(data, industry)
        links += link
    return industry_list, links

In [48]:
get_industry_regulation_links("media", data)

(['Digital Media'],
 ['https://www.congress.gov/bill/100th-congress/senate-bill/2361'])

In [49]:
get_industry_regulation_links("healthcare", data)

(['Medical'],
 ['https://www.cdc.gov/phlp/php/resources/health-insurance-portability-and-accountability-act-of-1996-hipaa.html'])

In [50]:
get_industry_regulation_links("finance", data)

(['Finance'],
 ['https://www.ftc.gov/business-guidance/privacy-security/gramm-leach-bliley-act',
  'https://www.fdic.gov/resources/supervision-and-examinations/consumer-compliance-examination-manual/documents/8/viii-3-1.pdf'])

In [51]:
get_industry_regulation_links("school", data)

(['Education'],
 ['https://www.disabilityrightssc.org/wp-content/uploads/2021/08/Family-Educational-Rights-and-Privacy-Act-FERPA-2020.pdf'])